In [309]:
#Se importan los paquetes necesarios para los cálculos.
import pandas as pd #manejo de datos
import numpy as np #parquete de matemática
import scipy.stats as sp #paquete de estadísticas
from sqlalchemy import create_engine #consultas a sql
import sqlalchemy #manejo de sql
import sys #manejo del sistema para recibir argumentos
import warnings #
from sqlalchemy import text
warnings.filterwarnings("ignore")
import pyodbc

import funciones

In [310]:
#Conexión a la base de datos
server = '10.0.0.153'
user = "testipc2"
pas = "Abc$2020"
#user = input('Ingrese el usuario: ')
#pas = input('Ingrese la contraseña: ')
try:
    #driver = 'SQL Server Native Client 11.0'
    driver = 'ODBC Driver 17 for SQL Server'
    database_connection = f'mssql://{user}:{pas}@{server}/master?driver={driver}'
    engine = create_engine(database_connection)
    connection = engine.connect()
    print('Conexión exitosa')
except:
    print('Fallo en la conexión')

Conexión exitosa


In [311]:
#aqui se ingresa el año y mes del indice que se va a calcular

anio = int(input('Ingrese el año: '))
mes = int(input('Ingrese el mes: '))

#anio = 2024
#mes = 1
dec = 0

In [312]:
#aca se generan los nombres de los archivos que tienen el indice anterior y el indice actual
excel_indantmes = funciones.excel_mesant(mes)
excel_indantanio = funciones.excel_anioant(anio, mes)
excel_indact01 = 'indprod'
excel_indact02 = '.xlsx'

excel_indant = f"{excel_indact01}_{excel_indantmes}_{excel_indantanio}{excel_indact02}"
excel_indactual = f"{excel_indact01}_{mes}_{anio}{excel_indact02}"

#excel_indactual
excel_indant

'indprod_4_2024.xlsx'

In [313]:
#Se seleccionan los códigos de los artículos, los demás códigos pueden obtenerse a partir de éstos.
def Codigos(connect):
    querycod = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    
    return pd.read_sql(querycod, connect, index_col = 'ArtCod')

codigos = Codigos(connection)
#codigos

In [314]:
#Se define una función para hallar cualquier conjunto de códigos.
#le asocia a la categoría su número en la base de datos
di = {'Art':6,'Gba':5,'Sub':4,'Gru':3,'Agr':2,'Div':1}
#le asocia a cada categoría la longitud de su código
leng = {'Div' : 2, 'Agr' : 3, 'Gru' : 4, 'Sub' : 5, 'Gba' : 7, 'Art': 9}

#Se seleccionan los códigos de los artículos, los demás códigos pueden obtenerse a partir de éstos.
def Nombres(datos, cod : str, connect = connection, dict = di):
    ''' 
    Esta función retorna un dataframe con los nombres y códigos de una categoría específica.

    Args:
    cod (DataFrame) : El dataframe con los códigos de una categoría dada, puede ser de artículos, gasto báscico, división, etc
    ind (str) : El código de categoría que deseamos. Gba : Gasto Básico, Sub : Subgrupo, Gru : Grupo, Agr : Agrupación, Div : División
    '''
    if cod == 'Art':
        querycod = text(f"SELECT DISTINCT ArtCod, ArtNom FROM IPC2010_RN.dbo.IPC007")
    else:
        querycod = text(f'SELECT DISTINCT {cod}Cod, {cod}Nom FROM IPC2010_RN.dbo.IPCM0{dict[cod]}')

    nombres = pd.read_sql(querycod, connect, index_col = cod+'Cod')
    return pd.merge(nombres.reset_index(), datos.reset_index(), on = cod+'Cod',  how = 'inner').set_index(['RegCod',cod+'Cod'])

def CodigosCat(ind :str, cod, len = leng):
    ''' 
    Esta función retorna un dataframe con los códigos de una categoría específica.

    Args:
    cod (DataFrame) : El dataframe con los códigos de una categoría dada, puede ser de artículos, gasto báscico, división, etc
    ind (str) : El código de categoría que deseamos. Gba : Gasto Básico, Sub : Subgrupo, Gru : Grupo, Agr : Agrupación, Div : División
    '''
    return pd.DataFrame([x[:len[ind]] for x in cod.index],columns=[ind+'Cod']).drop_duplicates().set_index(ind+'Cod')

In [315]:
#aqui se empieza la parte de la conexion con el otro server
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [316]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [317]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas01 = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful")
except Exception as e:
    print(f"Error: {e}")

Connection successful


Esta sección obtiene el dataframe boletas02 que contienen todos los datos con el formato que se utiliza para realizar el calculo del IPC

In [318]:
boletas02 = boletas01
boletas02['codigo_articulo'] = boletas01['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones
boletas02['decada'] = boletas02['decada'].replace(funciones.decadas)
boletas02['nt_tipo'] = boletas02['nt_tipo'].replace({351761:'S', 351762:'S'})
boletas02['nt_tipo'] = boletas02['nt_tipo'].fillna('N')
boletas02['cantidad_anterior'] = pd.to_numeric(boletas02['cantidad_anterior'], errors='coerce')
boletas02['cantidad_actual'] = pd.to_numeric(boletas02['cantidad_actual'], errors='coerce')
boletas02['precio_anterior'] = pd.to_numeric(boletas02['precio_anterior'], errors='coerce')
boletas02['precio_actual'] = pd.to_numeric(boletas02['precio_actual'], errors='coerce')
boletas02['decada'] = pd.to_numeric(boletas02['decada'], errors='coerce')
boletas02['region'] = boletas02['region'].astype(str)
boletas02['codigo_articulo'] = boletas02['codigo_articulo'].astype(str)
boletas02 = boletas02[['region', 'codigo_articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]
boletas02.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR', 'decada':'PerSem'}, inplace=True)
boletas02['RegCod'] = boletas02['RegCod'].replace({'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})

boletas02

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
0,01,053110202,34-5-2024,1.0,1.0,145.00,145.00,N,1
1,01,031220302,36-5-2024,1.0,1.0,250.00,250.00,N,1
2,01,072110201,42-5-2024,1.0,1.0,375.00,375.00,N,1
3,01,041110103,43-5-2024,1.0,1.0,1700.00,1700.00,N,1
4,01,3211051,52-5-2024,1.0,1.0,279.99,279.99,N,1
...,...,...,...,...,...,...,...,...,...
71312,08,4542021,24184-5-2024,0.0,1.0,0.00,5.00,N,3
71313,08,7322021,24197-5-2024,0.0,1.0,0.00,406.00,N,3
71314,08,093210101,24228-5-2024,0.0,1.0,0.00,35.00,N,3
71315,08,093210101,24228-5-2024,0.0,1.0,0.00,45.00,N,3


Esta sección obiene los precios que deben ser imputados para el mes, los articulos imputados son aquellos de la base 2010 que no aparecen en la base 2023 y por lo tanto "no se cotizan" 

In [319]:
boletas03 = boletas01
boletas03['codigo_articulo'] = boletas01['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones
boletas03 = boletas03[['codigo_articulo', 'articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]
artimput = boletas03.rename(columns={'codigo_articulo':'ArtCod'})
artimput = artimput[artimput['ArtCod'].str.len() == 9]
#se comparan con la lista de codigos de los 441 productos que conforman la canasta basica
codigos01 = codigos 
codigos01 = codigos01.reset_index()
artimput = codigos01[~codigos01['ArtCod'].isin(artimput['ArtCod'])]
para_imputar = artimput

In [320]:
#crea un dataframe con los articulos recodificados a la base 2010
similar = boletas02[boletas02['ArtCod'].str.len() == 9]
#similar

In [321]:
#se importan los precios del mes de diciembre de 2023 que son los ultimos precios que se conocen de la base 2010
boldic2023 = pd.read_excel("boletadic2023.xlsx")
boldic2023['ArtCod'] = boldic2023['ArtCod'].astype(str)
boldic2023.loc[boldic2023['ArtCod'].str.len() == 8, 'ArtCod'] = '0' + boldic2023['ArtCod']
boldic2023['RegCod'] = boldic2023['RegCod'].astype(str)
boldic2023['RegCod'] = boldic2023['RegCod'].replace({'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})

In [322]:
#se obtienen los productos que son "similares" a los productos imputados
#se hacen los ajustes necesarios al dataframe que contiene la lista de productos imputados
para_imputar['ArtCod'] = para_imputar['ArtCod'].astype(str)
para_imputar.loc[para_imputar['ArtCod'].str.len() == 8, 'ArtCod'] = '0' + para_imputar['ArtCod']

#se genera un dataframe que corta el codigo del articulo para poder evaluarlos a nivel de subgrupo
ref_similar = pd.DataFrame(para_imputar['ArtCod'].str[:-2])

In [323]:
#aqui voy a usar los datos de referencia similares para sacar de la base de datos del mes los articulos de referencia
ref_sim = ref_similar['ArtCod']
#ref_sim01 = ref_similar01['ArtCod']
ref_sim02 = para_imputar['ArtCod'] #este sirve para tener una serie de referencia con el string de codigo de articulo completo

#se obtienen los articulos que se encuentran dentro de un grupo similar para la base de datos del mes actual
similar01 = similar[similar['ArtCod'].str.startswith(tuple(ref_sim))]
#similar01

In [324]:
#se calcula la variacion para los articulos similares del mes actual
similar02 = similar01
similar02["variacion"] = ((similar02['ArtPac']*similar02['UraChi'])/(similar02['ArtPhi']*similar02['UreCan']))
similar02["variacion"] = similar02["variacion"].replace ([0, np.NaN,np.inf ], 1)

#similar02

In [325]:
#se obtienen un dataframe que contiene solamente el codigo del articulo y la variacion
variacion = similar02
variacion = similar02[['ArtCod', 'variacion']]
#variacion

In [326]:
#se calcula la variacion acumulada para los grupos de los productos que son similares a los imputados
var_ac = pd.DataFrame(variacion.groupby(ref_sim02)['variacion'].agg(sp.gmean)).reset_index()
#variacion[variacion['ArtCod'].str.startswith(tuple(ref_sim01))]
#var_ac

In [327]:
#se obtienen los precios del mes anterior para los articulos a imputar
#como esto es diciembre el precio a usar es el precio actual de diciembre
precioant = boldic2023[boldic2023['ArtCod'].str.startswith(tuple(ref_sim))]
funcionagg = {'ArtPac': 'median', 'ArtPhi': 'median', 'UraChi': 'median', 'UreCan': 'median', 'RegCod': 'first', 'BolNum': 'first', 'ArtCR': 'first', 'PerSem': 'first'}
pant_median = precioant.groupby('ArtCod').agg(funcionagg).reset_index() #utilice la mediana de los precios para no tener mucho problema con valores extremos
p_ant_imput = pd.merge(pant_median, para_imputar, on='ArtCod', how='inner') #se obtienen un dataframe con los precios a imputar
#p_ant_imput

In [328]:
#se obtiene un dataframe con los precios a imputar y la variacion acumulada anteriormente
#como no siempre estan todos los que se van a imputar en la boleta de diciembre para los precios anteriores entonces voy a llenar con 1
p_imput = pd.merge(p_ant_imput, var_ac, on='ArtCod', how='left')
p_imput['variacion'].fillna(1, inplace=True)
p_imput['ArtPimpt'] = (p_imput['ArtPac']*p_imput['variacion']) #se calcula el precio imputado

In [329]:
pimput = p_imput
pimput['ArtPac'] = pimput['ArtPimpt']
pimput.drop(columns=['ArtPimpt'], inplace=True)
#pimput

In [330]:
duplicated_rows = []
for i in range(1, 9):
    duplicated_rows.append(pimput.assign(RegCod=str(i).zfill(2)))

p_imp = pd.concat(duplicated_rows, ignore_index=True)
p_imp['RegCod'] = p_imp['RegCod'].astype(str)
p_imp.drop(columns=['variacion'], inplace=True)
desired_order = ['RegCod', 'ArtCod', 'BolNum', 'UraChi', 'UreCan', 'ArtPhi', 'ArtPac', 'ArtCR', 'PerSem']
p_imp = p_imp[desired_order]
p_imp

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
0,01,011110102,313510,460.0,460.0,5.00,5.00,N,1
1,01,011130104,313550,161.0,166.0,3.60,3.60,N,1
2,01,011130301,313519,1.0,1.0,10.00,10.00,N,1
3,01,011230102,313506,460.0,460.0,17.00,17.00,N,1
4,01,011230103,313506,460.0,460.0,14.50,15.00,N,1
...,...,...,...,...,...,...,...,...,...
651,08,122210101,313518,1.0,1.0,139.50,156.50,N,1
652,08,122210201,313778,1.0,1.0,345.00,350.00,N,1
653,08,122210301,313518,1.0,1.0,60.00,59.00,N,1
654,08,123110103,313793,1.0,1.0,182.27,182.27,N,1


In [331]:
boletas = boletas02[boletas02['ArtCod'].str.len() == 9]
boletas = pd.concat([boletas, p_imp])
boletas.set_index(['RegCod', 'ArtCod', 'BolNum'], inplace=True)
boletas.sort_index(inplace=True)
boletas

UraChi  UreCan   ArtPhi   ArtPac     ArtCR  \
RegCod ArtCod    BolNum                                                     
01     011110101 1035-5-2024    440.0   400.0     7.65     7.65  351763.0   
                 1104-5-2024    400.0   400.0     7.00     7.50         N   
                 113-5-2024     400.0   400.0     7.75     7.20         N   
                 1175-5-2024    400.0   400.0     6.15     6.80         N   
                 1220-5-2024    400.0   400.0     7.65     7.65         N   
...                               ...     ...      ...      ...       ...   
08     125110401 16027-5-2024     1.0     1.0  3800.00  3800.00         N   
                 16052-5-2024     1.0     1.0  3800.00  3800.00         N   
                 16435-5-2024     1.0     1.0  2800.00  2800.00         N   
                 16444-5-2024     1.0     1.0  3800.00  3800.00         N   
                 16551-5-2024     1.0     1.0  2000.00  2000.00         N   

                               PerSem  
RegCod ArtCod    BolNum                
01     011110101 1035-5-2024        1  
                 1104-5-2024        2  
                 113-5-2024         1  
                 1175-5-2024        2  
                 1220-5-2024        1  
...                               ...  
08     125110401 16027-5-2024       1  
                 16052-5-2024       3  
                 16435-5-2024       1  
                 16444-5-2024       1  
                 16551-5-2024       3  

[63966 rows x 6 columns]

In [332]:
boletas04 = boletas01[['codigo_articulo', 'articulo']].rename(columns={'codigo_articulo':'ArtCod'})
no_recod = boletas04[boletas04['ArtCod'].str.len() != 9]
no_recod

,ArtCod,articulo
4,3211051,Sandalias para hombre
15,1113052,Roscas
16,1113053,Campechanas y Lenguas
18,1113055,Donas
19,1113061,"Milhojas, Pastelitos"
...,...,...
71307,11112084,Ensalada de verduras preparadas
71309,1174032,Cebollas amarillas
71311,1194011,"Chile seco entero (guaque, pasa, etc.)"
71312,4542021,Manojo de ocote


In [333]:
prod_norecod = no_recod.groupby('ArtCod').value_counts().reset_index(name='Counts')
prod_norecod.sort_values('Counts', ascending=False, inplace=True)

#prod_norecod[prod_norecod['Counts'] == 147]

In [334]:
#se guardan los nombres de los productos que no se pueden recodificar a la base anterior
norecodname01 = 'sinrecodificar'
norecodname02 = '.xlsx'
norecodname = f"{norecodname01}_{mes}_{anio}{norecodname02}"

prod_norecod.to_excel(norecodname)

norecodname

'sinrecodificar_5_2024.xlsx'

In [335]:
#esta es la parte de los outliers
ignorar = False
if ignorar:
    ig = pd.read_excel(f'IGNORAR{anio}-{mes}.xlsx', dtype={f'ArtCod':str,f'RegCod':str})
    ig = ig.set_index(['RegCod','ArtCod','BolNum'])
else:
    ig = []

In [336]:
#Funcion para calcular las variaciones de cada cotización
def Variaciones(boleta, decada, outliers, ignorar = False):
    ''' 
    Esta función calcula las variaciones de la decada ingresada dado un arreglo de boletas y una decada
    
    Argumentos:
    boletas (DataFrame) : Un dataframe de las cotizaciones con las columnas RegCod, ArtCod, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem y el multiíndice (RegCod, ArtCod)
    dec (int) : La década en la que se va a calcular el índice, 

    '''
    boletas = boleta.copy()
    if decada == 0:
        semana = [1,2,3]
    elif decada == 12:
        semana = [1,2]
    elif decada == 23:
        semana =[2,3]
    else:
        semana = [decada]

    boletas = boletas.loc[boletas['PerSem'].isin(semana)]
    
    Variacion = pd.DataFrame((boletas['ArtPac']*boletas['UraChi'])/(boletas['ArtPhi']*boletas['UreCan']), columns = ['Var'])
    Variacion = Variacion.replace([np.nan,0,np.inf],1)
    Variacion.loc[boletas['ArtCR'] == 'S'] = 1.0
    if ignorar:
            Variacion.loc[Variacion.index.isin(outliers.index), 'Var'] = 1.0
    return Variacion

In [337]:
variaciones = Variaciones(boletas, dec, ig, ignorar)
variaciones

Var
RegCod ArtCod    BolNum                
01     011110101 1035-5-2024   1.100000
                 1104-5-2024   1.071429
                 113-5-2024    0.929032
                 1175-5-2024   1.105691
                 1220-5-2024   1.000000
...                                 ...
08     125110401 16027-5-2024  1.000000
                 16052-5-2024  1.000000
                 16435-5-2024  1.000000
                 16444-5-2024  1.000000
                 16551-5-2024  1.000000

[63966 rows x 1 columns]

In [338]:
def Periodos(connect):
        '''
        Llama las periodicidades de todos los artículos
        Args:
        connect (Connection) : conexión a la base de datos
        '''
        subquery = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 \
                WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
        query = text(f"SELECT ArtCod, \
                CASE \
                WHEN ArtPRC = 5 THEN  6\
                WHEN ArtPRC = 6 THEN  12\
                WHEN ArtPRC = 1 THEN  1\
                WHEN ArtPRC = 2 THEN  2\
                WHEN ArtPRC = 3 THEN  3\
                WHEN ArtPRC = 4 THEN  4\
                ELSE 1\
                END AS ArtPRC\
                FROM IPC2010_RN.dbo.IPC007 \
                WHERE ArtCod IN \
                ({subquery})\
                ORDER BY ArtCod")
        indice = 'ArtCod'
        return pd.read_sql(query, connect, index_col=indice)

In [339]:
#Función para calcular las variaciones acumuladas de los 441 artículos
def VarAcum(var,per, cod): #esta es la funcion que ellos llaman VarFlex
    ''' 
    Esta función calcula los índices relativos (variación acumulada) de los 441 artículos para todas las regiones y retorna un arreglo con índices (region, artículo). A los artículos que no fueron cotizados les asigna variación 1

    Args
    var (DataFrame) : un dataframe con las variaciones de las fuentes con multiíndice RegCod, ArtCod
    per (DataFrame) : dataframe con las periodicidades de los artículos con índice ArtCod
    cod (DataFrame) : un dataframe con los 441 códigos de los artículos
    '''

    agrupacion = var.groupby(level = ['RegCod','ArtCod']).agg(sp.gmean)
    periodicidad = agrupacion.join(per.astype(float), how = 'outer')
    raiz = pd.DataFrame(periodicidad['Var']**(1/(periodicidad['ArtPRC'])), columns = ['Var']).sort_index()
    arreglo = pd.DataFrame(['01','02','03','04','05','06','07','08'], columns= ['RegCod'])
    Codigos = cod.copy()
    Codigos['ArtCod'] = Codigos.index
    codigos2 = Codigos.merge(arreglo, how = 'cross')
    codigos2 = codigos2.set_index(['RegCod','ArtCod'])
    codigos2['Var'] = raiz['Var']
    codigos2 = codigos2.replace(np.nan, 1)
    return codigos2

In [340]:
periodos = Periodos(connection)
Varac_prod = VarAcum(variaciones, periodos, codigos)
Varac_prod

,,Var
RegCod,ArtCod,
01,011110101,1.013473
02,011110101,1.001866
03,011110101,1.002075
04,011110101,0.988702
05,011110101,0.999901
...,...,...
04,125110501,1.000000
05,125110501,1.000000
06,125110501,1.000000


In [341]:
#Se agregan las variaciones del armado nacional
def FormacionNacional(indart):
    lista = ['071110101','071110102','071110103','072420201','073210101','083110101','083110202','083110301','083110302',\
                    '083110401','093210201','093310101','095110201','123110101','123110102','123110103','123110104','124110101','125110101','125110501']
    indices = indart.copy()
    for index in lista:
        try:
            indices.loc[(indart.index.get_level_values('ArtCod') == index),'Var'] = indart.loc['01', index]['Var']
        except KeyError:
            pass
    return indices

In [342]:
Varac_prod_01 = FormacionNacional(Varac_prod).sort_index() #variacion acumulada
Varac_prod_01

Var
RegCod ArtCod             
01     011110101  1.013473
       011110102  1.000000
       011110103  1.005822
       011120101  0.984028
       011120102  0.999530
...                    ...
08     125110101  1.000000
       125110201  0.941036
       125110301  1.000000
       125110401  1.000000
       125110501  1.000000

[3528 rows x 1 columns]

In [343]:
#Función para llamar la ponderación de cualquier categoría
def CatPon(cod : str, dic, connect):
    if cod == 'Art':
        subquery = text(f"SELECT DISTINCT {cod}Cod FROM IPC2010_01_RN.dbo.IPCPH6 \
                    WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    else:
        subquery = text(f"SELECT DISTINCT {cod}Cod FROM IPC2010_01_RN.dbo.IPCM0{dic[cod]}")
    
    query = text(f'SELECT RegCod, {cod}Cod, {cod}Pon\
        FROM  IPC2010_RN.dbo.IPCP0{dic[cod]}\
        WHERE {cod}Cod IN ({subquery}) \
        ORDER BY RegCod, {cod}Cod')              
    return pd.read_sql(query, connect, index_col=['RegCod', cod+'Cod'])

In [344]:
#Función de cálculo de índices (relativos o absolutos)
def CatInd(cod :str, Varac_prod_01, pond, len = leng, codigo = codigos):
    pond = pond.copy()
    pond.columns = ['Pon']
    index = Varac_prod_01.copy()
    index.columns = ['Ind']
    pond[cod+'Cod'] = pond.index.get_level_values(1).map(lambda x: x[:len[cod]])
    pond = pond.set_index([pond.index.get_level_values(0), pond[cod+'Cod'], pond.index.get_level_values(1)]).drop(columns= [cod+'Cod'])
    pond[cod+'Pon'] = pond.groupby(level = ['RegCod', cod+'Cod']).sum()
    pond[cod+'Pon']=pond[cod+'Pon'].replace(0,1)
    cont = pd.DataFrame(index['Ind']*pond['Pon']/pond[cod+'Pon'], columns = [cod+'Ind'])
    cont[cod+'Cod'] = cont.index.get_level_values(1).map(lambda x: x[:len[cod]])
    cont = cont.set_index([cont.index.get_level_values(0), cont[cod+'Cod'], cont.index.get_level_values(1)]).drop(columns= [cod+'Cod'])
    catind = cont.groupby(level = ['RegCod', cod+'Cod']).sum()

    if cod == 'Gba':
        arreglo = pd.DataFrame(['01','02','03','04','05','06','07','08'], columns= ['RegCod'])
        cods = CodigosCat(cod, codigo)
        Codigos = cods.copy()
        Codigos[cod+'Cod'] = Codigos.index
        codigos2 = Codigos.merge(arreglo, how = 'cross')
        codigos2 = codigos2.set_index(['RegCod',cod+'Cod'])
        codigos2[cod+'Ind'] = catind[cod+'Ind']
        codigos2 = codigos2.replace(np.nan, 1)
        return codigos2
    else:
        return catind

In [345]:
CatPon('Gba', di, connection)

GbaPon
RegCod GbaCod          
00     0111101  0.63244
       0111201  0.17119
       0111202  1.33671
       0111203  0.93761
       0111301  2.61053
...                 ...
08     1251101  0.11386
       1251102  0.00713
       1251103  0.05303
       1251104  0.16740
       1251105  0.00000

[2511 rows x 1 columns]

In [346]:
pond = CatPon('Art', di, connection) #ponderaciones 
pond

ArtPon
RegCod ArtCod              
00     011110101   64.74764
       011110102   20.38296
       011110103   14.86940
       011120101   90.25644
       011120102    9.74356
...                     ...
08     125110101  100.00000
       125110201  100.00000
       125110301  100.00000
       125110401  100.00000
       125110501    0.00000

[3969 rows x 1 columns]

In [347]:
#Calculamos los índices relativos (variación) de gasto básico de las 8 regiones
Varprod02 = CatInd('Gba', Varac_prod_01, pond, leng)
Varprod02

,,GbaInd
RegCod,GbaCod,
01,0111101,1.008500
02,0111101,1.002330
03,0111101,1.003620
04,0111101,0.993114
05,0111101,1.000582
...,...,...
04,1251105,1.000000
05,1251105,0.000000
06,1251105,1.000000


In [348]:
#extraemos los indices anteriores
indant01 = pd.read_excel(excel_indant)
indant01.drop(indant01.columns[0], axis=1, inplace=True)
indant01['RegCod'] = indant01['RegCod'].astype(str)
indant01['RegCod'] = indant01['RegCod'].replace({'0':'00','1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})
indant01['GbaCod'] = indant01['GbaCod'].astype(str)
indant01['GbaCod'] = indant01['GbaCod'].apply(lambda x: '0' + x if len(x) == 6 else x)
indant01 = indant01[indant01['RegCod'] != '00']
indant01 = indant01.set_index(['RegCod', 'GbaCod'])
indant = indant01
indant

GbaInd
RegCod GbaCod             
01     0111101  183.069791
       0111201  252.405548
       0111202  237.217041
       0111203  153.478091
       0111301  174.927678
...                    ...
08     1251101  119.542285
       1251102   87.622476
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2232 rows x 1 columns]

In [349]:
#Función para extraer índices anteriores de cualquier categoría
def IndAnt(anio : int, mes : int, cod : str, conectar, dic = di, ano = False):
    if ano:
        mesant = mes
        anioant = anio - 1
    else:
        if mes == 1:
            anioant = anio -1
            mesant = 12
        else:
            anioant = anio
            mesant = mes-1

    query = text(f'SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_01_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_02_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_03_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_04_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_05_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_06_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_07_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_08_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        ORDER BY RegCod, {cod}Cod')
    return pd.read_sql(query, conectar, index_col=['RegCod',cod+'Cod'])

In [350]:
indanioant = IndAnt(anio, mes, 'Gba', connection, ano = True)
indanioant

GbaInd
RegCod GbaCod             
01     0111101  173.518909
       0111201  239.402195
       0111202  244.411638
       0111203  156.290167
       0111301  178.373463
...                    ...
08     1241101   96.475341
       1251101  120.596493
       1251102  115.187051
       1251103  113.873077
       1251104  117.294604

[2185 rows x 1 columns]

In [351]:
indprod = pd.DataFrame(indant['GbaInd']*Varprod02['GbaInd'], columns= ['GbaInd']).replace(np.nan, 0)
indprod #este seria el indice de productos actuales

GbaInd
RegCod GbaCod             
01     0111101  184.625973
       0111201  249.005742
       0111202  228.582725
       0111203  154.042331
       0111301  176.202850
...                    ...
08     1251101  119.542285
       1251102   82.455907
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2232 rows x 1 columns]

In [352]:
#Función para llamar las ponderaciones de cada región
def RegPon(connect):
    query = text(f'SELECT RegCod, RegPon \
        FROM IPC2010_RN.dbo.IPC001 WHERE RegCod != 0')            
    return pd.read_sql(query, connect, index_col= 'RegCod')

In [353]:
#Función para calcular el índice por república
def IndRep(cod :str, index, regpond, indponde):
    indices = index.copy()
    regpon = regpond.copy()
    indpon = indponde.copy()
    indreg = indices.loc[(indices.index.get_level_values(0) != '00')]
    indpond = indpon.loc[(indpon.index.get_level_values(0) != '00')]
    reppon = pd.DataFrame(indpond[cod+'Pon']*regpon['RegPon'],columns = ['Pon'])
    sumas = reppon.groupby(level =cod+ 'Cod').sum()
    regcont = pd.DataFrame(indreg[cod+'Ind']*reppon['Pon']/sumas['Pon'],columns = [cod+'Ind'])
    indrep = regcont.groupby(level = [cod+'Cod']).sum()
    indrep['RegCod'] = '00'
    indrep = indrep.set_index([indrep['RegCod'], indrep.index]).drop(columns = ['RegCod'])
    indices.loc[indices.index.get_level_values(0) == '00'] = indrep
    return pd.concat([indrep, indreg])

In [354]:
#ponderaciones de la region a nivel republica
regpon = RegPon(connection)
regpon

,RegPon
RegCod,
01,37.34790
02,5.40404
03,7.43584
04,6.53469
05,10.96698
06,18.53248
07,9.66149
08,4.11658


In [355]:
#ponderaciones de los productos a nivel republica
gbapon = CatPon('Gba', di, connection)
gbapon

GbaPon
RegCod GbaCod          
00     0111101  0.63244
       0111201  0.17119
       0111202  1.33671
       0111203  0.93761
       0111301  2.61053
...                 ...
08     1251101  0.11386
       1251102  0.00713
       1251103  0.05303
       1251104  0.16740
       1251105  0.00000

[2511 rows x 1 columns]

In [356]:
#indice de productos del mes anterior a nivel nacional
indprod_repant = IndRep('Gba', indant, regpon, gbapon)
indprod_repant

GbaInd
RegCod GbaCod             
00     0111101  165.139014
       0111201  228.089436
       0111202  341.092589
       0111203  143.618991
       0111301  189.409389
...                    ...
08     1251101  119.542285
       1251102   87.622476
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2511 rows x 1 columns]

In [357]:
#indice prodcutos a nivel republica del año anterior
indprod_anioant = IndRep('Gba', indanioant, regpon, gbapon)
indprod_anioant

GbaInd
RegCod GbaCod             
00     0111101  159.117209
       0111201  217.497599
       0111202  345.763061
       0111203  143.050768
       0111301  191.401321
...                    ...
08     1241101   96.475341
       1251101  120.596493
       1251102  115.187051
       1251103  113.873077
       1251104  117.294604

[2464 rows x 1 columns]

In [358]:
#indice productos a nivel republica actual
indprod_rep = IndRep('Gba', indprod, regpon, gbapon)
indprod_rep

GbaInd
RegCod GbaCod             
00     0111101  165.882225
       0111201  226.968521
       0111202  340.651583
       0111203  143.800808
       0111301  190.157787
...                    ...
08     1251101  119.542285
       1251102   82.455907
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2511 rows x 1 columns]

In [359]:
indprod_rep01 = indprod_rep.reset_index()
indprod_rep01

,RegCod,GbaCod,GbaInd
0,00,0111101,165.882225
1,00,0111201,226.968521
2,00,0111202,340.651583
3,00,0111203,143.800808
4,00,0111301,190.157787
...,...,...,...
2506,08,1251101,119.542285
2507,08,1251102,82.455907
2508,08,1251103,114.954121
2509,08,1251104,117.797094


In [360]:
indprod_rep01.to_excel(excel_indactual)

In [361]:
''' 
Aquí se pueden calcular los índices de cualquier categoría, sólo se debe cambiar el argumento Div (división) por el argumento Agr, Gru, Sub, Gba dependiendo del índice que se desee
'''
cat = 'Div'
inddiv = CatInd(cat, indprod_rep, gbapon)
pondiv = CatPon(cat, di, connection)
inddiv_rep = IndRep(cat, inddiv, regpon, pondiv)
inddiv_rep #indice de division a nivel republica actual

DivInd
RegCod DivCod            
00     01      293.874119
       02      131.221292
       03      120.462597
       04      129.094332
       05      131.510119
...                   ...
08     08      114.072439
       09      117.336512
       10      106.772716
       11      144.396063
       12      125.785588

[108 rows x 1 columns]

In [362]:
def IPC(index, pond):
    indices = index.copy()
    indices.columns = ['Ind']
    pon = pond.copy()
    pon.columns = ['Pon']
    return pd.DataFrame(indices['Ind']*pon['Pon']/100, columns = ['IPC']).groupby(level = ['RegCod']).sum()

In [363]:
'''esto seria el IPC actual'''
#IPC nacional actual
ipc_actual = IPC(inddiv, pondiv)
ipc_actual

,IPC
RegCod,
00,179.069181
01,145.150481
02,277.613748
03,163.529036
04,281.260181
05,159.262191
06,161.343967
07,253.848174
08,180.350470


In [364]:
#IPC nacional mes pasado
ipc_anterior = IPC(indprod_repant, gbapon)
ipc_anterior

,IPC
RegCod,
00,177.350852
01,145.665131
02,274.263047
03,163.403468
04,273.984794
05,159.118716
06,160.571842
07,243.020012
08,179.385038


In [365]:
ipc_anioant = IPC(indprod_anioant, gbapon)
ipc_anioant

,IPC
RegCod,
00,169.371309
01,141.060583
02,254.797038
03,160.158303
04,237.927025
05,155.241098
06,154.975695
07,234.146983
08,172.323230


Variaciones

In [366]:
#variacion intermensual
(ipc_actual-ipc_anterior)*100/ipc_anterior

,IPC
RegCod,
00,0.968886
01,-0.353310
02,1.221711
03,0.076846
04,2.655398
05,0.090169
06,0.480859
07,4.455667
08,0.538190


In [367]:
(ipc_actual-ipc_anioant)*100/ipc_anioant

,IPC
RegCod,
00,5.725805
01,2.899391
02,8.954857
03,2.104626
04,18.212793
05,2.590225
06,4.109207
07,8.414028
08,4.658246


Incidencias por division

In [368]:
def Incidencias(index, indexant, pond,ipcante):
    pon = pond.copy()
    pon.columns = ['Pon']
    indicesant = indexant.copy()
    indicesant.columns = ['Ind']
    indices = index.copy()
    indices.columns = ['Ind']
    return pd.DataFrame(pon['Pon']*(indices['Ind']-indicesant['Ind'])/ipcante['IPC'],columns= ['Incid'])

In [369]:
inddiv_repant = CatInd('Div', indprod_repant, gbapon)

In [370]:
incid_divrep = Incidencias(inddiv_rep, inddiv_repant, pondiv, ipc_anterior)
incid_divrep.loc[incid_divrep.index.get_level_values('RegCod') == '00']

Incid
RegCod DivCod              
00     01      1.060522e+00
       02      3.256408e-04
       03      3.153794e-03
       04     -6.324613e-03
       05     -2.393226e-03
       06      1.448372e-03
       07     -1.432550e-01
       08     -1.410747e-03
       09      1.673130e-02
       10      2.291438e-07
       11      1.797673e-02
       12      2.220594e-02

In [371]:
incid_divrep.sort_values('Incid')

,,Incid
RegCod,DivCod,
01,07,-0.281195
00,07,-0.143255
01,01,-0.142982
03,07,-0.129536
05,07,-0.097704
...,...,...
06,01,0.515085
00,01,1.060522
02,01,1.232434


Incidencia productos

In [372]:
incid_prodrep = Incidencias(indprod_rep, indprod_repant, gbapon, ipc_anterior)
incid_prodrep.loc[incid_prodrep.index.get_level_values('RegCod') == '00']

Incid
RegCod GbaCod           
00     0111101  0.002650
       0111201 -0.001082
       0111202 -0.003324
       0111203  0.000961
       0111301  0.011016
...                  ...
       1251101  0.000000
       1251102 -0.000661
       1251103  0.000000
       1251104  0.000000
       1251105  0.000000

[279 rows x 1 columns]

In [373]:
incid_prodrep.loc[incid_prodrep['Incid'].notnull()]

Incid
RegCod GbaCod           
00     0111101  0.002650
       0111201 -0.001082
       0111202 -0.003324
       0111203  0.000961
       0111301  0.011016
...                  ...
08     1251101  0.000000
       1251102 -0.000205
       1251103  0.000000
       1251104  0.000000
       1251105  0.000000

[2511 rows x 1 columns]

In [374]:
ipc_actualreg = ipc_actual.loc[ipc_actual.index != '00']
ipc_antreg = ipc_anterior.loc[ipc_anterior.index != '00']
ipc_anttotal = ipc_anterior.loc['00','IPC']

In [375]:
#incid_reg = 
pd.DataFrame(regpon['RegPon']*(ipc_actualreg['IPC']-ipc_antreg['IPC'])/ipc_anttotal, columns = ['Incid'])
#incid_reg

,Incid
RegCod,
01,-0.108379
02,0.102099
03,0.005265
04,0.268070
05,0.008872
06,0.080684
07,0.589883
08,0.022409
